-------------------------------------------
Chains in LangChain
Outline-->

1.LLMChain
2.Sequential Chains
3.SimpleSequentialChain
4.SequentialChain
5.Router Chain
===================

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv, find_dotenv

load_dotenv(dotenv_path=r"C:\Users\hasan\Rafi_SAA\python_practices\LLM_Langchain\llm_openai\.env")


True

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

---------------------------
Simple LLM chain
===================

In [ ]:
chat = ChatOpenAI(
    model="openai/gpt-4o",
    openai_api_key=os.getenv("GPT_github_access_token"),
    openai_api_base="https://models.github.ai/inference"

)

In [6]:
prompt = ChatPromptTemplate.from_template("What is a good name for a company that makes {product}?")

In [ ]:
chain = LLMChain(llm=chat, prompt=prompt)

In [ ]:
product  = "sports shoes"
chain.run(product)

In [30]:
skin_tone = "light"
body_type = "skinny"
available_clothes = "t-shirts, jeans, sneakers, jackets, hats, scarves, belts, sunglasses, watches, gown, ties, blazers, skirts"
available_clothes = "".join(available_clothes)


In [34]:
prompt_fashion = ChatPromptTemplate.from_template("You are a fashion expert. Suggest short note for type of clothing based on the :{available_clothes}, my {skin_tone} skin tone , {body_type} body type.")


In [35]:
chain2 = LLMChain(llm= chat, prompt= prompt_fashion)
result2 = chain2.run({
    "available_clothes": available_clothes,
    "skin_tone": skin_tone,
    "body_type": body_type
})
print(result2)


Here’s a concise fashion guide tailored to your light skin tone and skinny body type:

1. **T-Shirts**: Opt for slim or fitted tees in pastel shades, jewel tones, or earthy colors to complement your skin. Vertical stripes or subtle patterns can add dimension.  
2. **Jeans**: Skinny or slim-fit jeans work well to enhance your proportions. Light-wash, dark-wash, or distressed denim adds versatility.  
3. **Sneakers**: Classic white sneakers or neutral shades are versatile, while pops of color add fun to casual looks.  
4. **Jackets**: Lightweight bomber jackets, cropped jackets, or tailored blazers in neutral or muted tones flatter your frame.  
5. **Hats**: Choose fedora, beanie, or baseball caps in earthy or pastel shades to suit your complexion.  
6. **Scarves**: Lightweight scarves in soft fabrics and muted patterns add a chic touch without overwhelming your frame.  
7. **Belts**: Slim belts in neutral colors or metallic finishes define your waist and add polish.  
8. **Sunglasses**:

In [ ]:
print(result2)

-------------------------------------
simple sequential Chain
=========================

In [11]:
from langchain.chains import SimpleSequentialChain
# Simple LLM chain

In [23]:
body_type = "athletic"
available_clothes = "t-shirts, jeans, sneakers, jackets, hats, scarves, belts, sunglasses, watches, gown, ties, blazers, skirts"
available_clothes = "".join(available_clothes)
print(available_clothes)

t-shirts, jeans, sneakers, jackets, hats, scarves, belts, sunglasses, watches, gown, ties, blazers, skirts


In [24]:
# first prompt template
prompt1 = ChatPromptTemplate.from_template("You are a fashion expert. Suggest a type of clothing based on the body type: {body_type}")

#chain1
chain1 = LLMChain(llm= chat, prompt = prompt1, output_key = "clothing_suggestion")

In [25]:

prompt2 = ChatPromptTemplate.from_template(
    "You are a fashion expert. Based on your suggestion and my {available_clothes}, make a suggestion. "
    "If no clothes are available, then suggest from your suggestion."
)

chain2 = LLMChain(llm= chat, prompt = prompt2)


In [ ]:
# prompt2.format(available_clothes=available_clothes)
# print(available_clothes)

t-shirts, jeans, sneakers, jackets, hats, scarves, belts, sunglasses, watches, gown, ties, blazers, skirts


In [26]:
overall_chain = SimpleSequentialChain(chains = [chain1, chain2], verbose = True)

In [28]:
result  = overall_chain.run(body_type)
print(result)



> Entering new SimpleSequentialChain chain...
For an athletic body type, the goal is often to create the illusion of curves and balance your naturally toned and straight silhouette. Here are some clothing suggestions that work well for athletic builds:

### Tops:
- **Peplum Tops:** Add volume to the waist and create a curvier appearance.
- **Wrap Tops:** Accentuate the waist and add a feminine shape.
- **Off-the-Shoulder Tops:** Highlight your shoulders and create softness.
- **Ruffled or Draped Details:** Add volume to your upper body.

### Bottoms:
- **High-Waisted Jeans or Pants:** Define your waist and create curves.
- **Wide-Leg Trousers or Palazzo Pants:** Balance a straight silhouette with flowy, voluminous bottoms.
- **A-Line Skirts:** Give the illusion of hips and add femininity.
- **Paper Bag-Waist Pants:** Add emphasis to your waist and create shape.

### Dresses:
- **Fit-and-Flare Dresses:** Highlight your toned upper body and add volume to your lower half.
- **Bodycon Dr

utilizing weather data from api to give as prompt to LLM

In [36]:
import requests

In [50]:
# 1. Fetch weather data dynamically
load_dotenv(dotenv_path=r"C:\Users\hasan\Rafi_SAA\python_practices\LLM_Langchain\llm_openai\.env")
base_url = "https://api.weatherapi.com/v1"
def forecast_weather(city):
    api_key = os.getenv("weather_api_key")
    url = f"{base_url}/forecast.json?key={api_key}&q={city}&days=1&aqi=no&alerts=no"
    data = requests.get(url).json()
    print(data)
    return data
    # forecast = data["forecast"]["forecastday"][0]["day"]
    # condition = forecast["condition"]["text"]
    # avg_temp = forecast["avgtemp_c"]
    # return f"{condition.lower()}, around {avg_temp}°C"

In [38]:
prompt_fashion_new = ChatPromptTemplate.from_template("""
You are a fashion expert.
Suggest a type of clothing suitable for the weather in {city} ({weather_info}),
considering that I have these clothes: {available_clothes},
my skin tone is {skin_tone},
and my body type is {body_type}.
""")

In [51]:
# 3. Call API and run chain
weather_info = forecast_weather("Barcelona")


{'location': {'name': 'Barcelona', 'region': 'Catalonia', 'country': 'Spain', 'lat': 41.3833, 'lon': 2.1833, 'tz_id': 'Europe/Madrid', 'localtime_epoch': 1760780617, 'localtime': '2025-10-18 11:43'}, 'current': {'last_updated_epoch': 1760779800, 'last_updated': '2025-10-18 11:30', 'temp_c': 19.2, 'temp_f': 66.6, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 4.3, 'wind_kph': 6.8, 'wind_degree': 94, 'wind_dir': 'E', 'pressure_mb': 1020.0, 'pressure_in': 30.12, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 83, 'cloud': 25, 'feelslike_c': 19.2, 'feelslike_f': 66.6, 'windchill_c': 20.1, 'windchill_f': 68.2, 'heatindex_c': 20.1, 'heatindex_f': 68.2, 'dewpoint_c': 14.8, 'dewpoint_f': 58.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.5, 'gust_mph': 4.9, 'gust_kph': 7.9, 'short_rad': 185.54, 'diff_rad': 53.73, 'dni': 868.63, 'gti': 0.0}, 'forecast': {'forecastday': [{'date': '2025-10-18', 'date_epoch': 1760745600

In [52]:
chain = LLMChain(llm=chat, prompt=prompt_fashion_new)

In [53]:
response = chain.run({
    "city": "Barcelona",
    "weather_info": weather_info,
    "available_clothes": "jeans, t-shirt, light jacket",
    "skin_tone": "olive",
    "body_type": "slim"
})

print(response)


Given the weather in Barcelona on October 18, 2025, with temperatures ranging between 18.2°C (64.8°F) and 20.9°C (69.7°F), and conditions varying between sunny and patchy rain, your wardrobe choices are well-suited. Here's how you can style your outfit to be both comfortable and chic:

### Outfit Recommendation:
1. **Jeans:**  
   Jeans are perfect for this weather, offering warmth during cooler moments while still breathable enough for midday temperatures. Choose a slim or straight-fit style to complement your slim body type.

2. **T-Shirt:**  
   A simple t-shirt works well as a base layer, especially since the weather is sunny for most of the day. Opt for neutral or earthy tones like beige, olive green, or soft brown to flatter your olive skin tone.

3. **Light Jacket:**  
   Since there’s a chance of patchy rain and humidity is relatively high, a light jacket is essential. A lightweight bomber or utility jacket in a complementary shade (like khaki, navy, or charcoal gray) will keep